# Q: Can Artificial Intelligence (AI) play games (like HTML5 Games similar to this - https://k4.games/)? If yes, how can you use concepts of computer vision to prove this and tool you need to use. 


In [ ]:
import cv2
import numpy as np
import gym
from stable_baselines3 import PPO
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

# Initialize Selenium WebDriver
driver = webdriver.Chrome()
driver.get("https://k4.games/")

# Function to capture game screenshot and identify player position
def get_screenshot_and_player(image):
  # Process the image (replace with specific detection logic)
  # Here's a basic example assuming the player is a red circle
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  lower_red = np.array([160, 100, 100])
  upper_red = np.array([180, 255, 255])
  mask = cv2.inRange(hsv, lower_red, upper_red)
  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  player_center = None
  # Find the largest contour (assuming it's the player)
  if len(contours) > 0:
    largest_contour = max(contours, key=cv2.contourArea)
    ((x, y), radius) = cv2.minEnclosingCircle(largest_contour)
    player_center = (int(x), int(y))
  return image, player_center

# Custom Gym environment
class GameEnv(gym.Env):
  def __init__(self):
    super(GameEnv, self).__init__()
    # Define action and observation space
    self.action_space = gym.spaces.Discrete(2)  # Example: 2 actions (left, right)
    self.observation_space = gym.spaces.Box(low=0, high=255, shape=(height, width, 3), dtype=np.uint8)
    # Initial values (update during reset)
    self.height = None
    self.width = None
    self.player_center = None

  def reset(self):
    driver.refresh()
    # Consider waiting for specific game element to reload
    # ...
    screenshot, self.player_center = get_screenshot_and_player(driver.get_screenshot_as_png())
    self.height, self.width = screenshot.shape[:2]
    self.observation_space = gym.spaces.Box(low=0, high=255, shape=(self.height, self.width, 3), dtype=np.uint8)
    return screenshot

  def step(self, action):
    # Explore alternative element selection for key presses
    # ...
    if action == 0:
      driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.ARROW_LEFT)
    else:
      driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.ARROW_RIGHT)

    time.sleep(0.1)  # Temporary sleep for demonstration

    obs, _ = get_screenshot_and_player(driver.get_screenshot_as_png())
    # Calculate reward based on game state (e.g., distance moved closer to goal)
    reward = 0  # Implement reward logic (consider player movement)
    # Determine done condition based on game termination (game over screen)
    done = False  # Implement done logic

    # Check if player moved closer or further from potential goal (replace with actual goal detection)
    if self.player_center is not None and obs is not None:
      new_player_center, _ = get_screenshot_and_player(obs)
      if new_player_center is not None:
        distance_before = cv2.norm(self.player_center)
        distance_after = cv2.norm(new_player_center)
        reward = distance_before - distance_after

    self.player_center = new_player_center
    return obs, reward, done, {}

# Create and train the model (assuming height and width are obtained)
env = GameEnv()
model = PPO('CnnPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

# Play the game with the trained mode

# Q: Is AI animation is possible? If yes, what kind of AI/ML tools can be used for making videos (like https://www.youtube.com/watch?v=ajKIsf4ncu0 ). Also, let us know how can we develop some basic tools for the same.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Create the generator
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_dim=100, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(784, activation='sigmoid'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

# Create the discriminator
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Compile the GAN
def build_gan(generator, discriminator):
    discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False
    gan_input = layers.Input(shape=(100,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = tf.keras.models.Model(gan_input, gan_output)
    gan.compile(optimizer='adam', loss='binary_crossentropy')
    return gan

generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Training the GAN
def train_gan(gan, generator, discriminator, epochs=10000, batch_size=128):
    (X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, valid)
        
        if epoch % 1000 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}] [G loss: {g_loss}]")
            save_images(generator, epoch)

def save_images(generator, epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, 100))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    fig, axs = plt.subplots(dim[0], dim[1], figsize=figsize)
    count = 0
    for i in range(dim[0]):
        for j in range(dim[1]):
            axs[i, j].imshow(generated_images[count, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            count += 1
    plt.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.close()

train_gan(gan, generator, discriminator)
